<a href="https://colab.research.google.com/github/Varsha-code-bit/INTERVIEW-QUESTIONS/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install wolta
!pip install ultralytics
!pip install transformers
!pip install torch torchvision
!pip install tensorflow
!pip install hyperopt

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
import cv2
import os
from glob import glob
from pathlib import Path
import zipfile

from wolta.visual_tools import get_extensions, dataset_size_same, dataset_ratio_same, examine_sizes, dir_split
from ultralytics import YOLO
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification
from torch import nn
from torch.optim import Adam
from tensorflow.keras import layers, Sequential

os.environ['WANDB_MODE'] = 'disabled'


In [23]:
# Paths
zip_path = '/content/dataset_ISL_.zip'
extract_path = '/content/isl_dataset'

# Extract ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Files extracted to:", extract_path)

# Check structure
for dirname, _, _ in os.walk(extract_path):
    print(dirname)


Files extracted to: /content/isl_dataset
/content/isl_dataset
/content/isl_dataset/j
/content/isl_dataset/u
/content/isl_dataset/q
/content/isl_dataset/e
/content/isl_dataset/c
/content/isl_dataset/o
/content/isl_dataset/f
/content/isl_dataset/raw
/content/isl_dataset/raw/j
/content/isl_dataset/raw/u
/content/isl_dataset/raw/q
/content/isl_dataset/raw/e
/content/isl_dataset/raw/c
/content/isl_dataset/raw/o
/content/isl_dataset/raw/f
/content/isl_dataset/raw/raw
/content/isl_dataset/raw/g
/content/isl_dataset/raw/k
/content/isl_dataset/raw/v
/content/isl_dataset/raw/p
/content/isl_dataset/raw/t
/content/isl_dataset/raw/w
/content/isl_dataset/raw/d
/content/isl_dataset/raw/s
/content/isl_dataset/raw/y
/content/isl_dataset/raw/r
/content/isl_dataset/raw/l
/content/isl_dataset/raw/n
/content/isl_dataset/raw/x
/content/isl_dataset/raw/h
/content/isl_dataset/raw/b
/content/isl_dataset/raw/a
/content/isl_dataset/raw/z
/content/isl_dataset/raw/i
/content/isl_dataset/raw/m
/content/isl_dataset/

In [24]:
from glob import glob
import os

extract_path = '/content/isl_dataset'  # Ensure extract_path is defined

# Use glob to find all files within the extracted directory and its subdirectories
all_paths = glob(os.path.join(extract_path, '**/*'), recursive=True)

# Filter the list to include only files
i_paths = [path for path in all_paths if os.path.isfile(path)]

print("Total images:", len(i_paths))

Total images: 37911


In [25]:
from wolta.visual_tools import get_extensions, dataset_size_same, dataset_ratio_same, examine_sizes

get_extensions(i_paths)
dataset_size_same(i_paths)
dataset_ratio_same(i_paths)
examine_sizes(i_paths)

temp_img = cv2.imread(i_paths[0])
height, width = temp_img.shape[:2]
print('ratio: {}'.format((width/height)))


Average Width x Height: 183.54844240457913 x 183.8288095803329
Biggest width is 1836 and the smallest width is 100
Biggest height is 2314 and the smallest height is 100
ratio: 1.0


In [29]:
raw_dir = '/content/isl_dataset/raw'
os.makedirs(raw_dir, exist_ok=True)

# Define d_paths by listing directories within extract_path, excluding 'raw' and 'data'
extract_path = '/content/isl_dataset'
d_paths = [os.path.join(extract_path, d) for d in os.listdir(extract_path) if os.path.isdir(os.path.join(extract_path, d)) and d not in ['raw', 'data']]


for d_path in d_paths:
    class_name = Path(d_path).name
    class_dir = os.path.join(raw_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

    # Use glob to find only files within the directory
    img_paths = glob(os.path.join(d_path, '*.*')) # Added .* to only match files with extensions
    for img_path in img_paths:
        img_name = Path(img_path).name
        img = cv2.imread(img_path)
        # Add a check to ensure img is not None before resizing
        if img is not None:
            img = cv2.resize(img, (224, 224))
            cv2.imwrite(os.path.join(class_dir, img_name), img)
        else:
            print(f"Warning: Could not read image file: {img_path}")

In [31]:
import shutil

data_dir = '/content/isl_dataset/data'

# Remove the data directory if it exists
if os.path.exists(data_dir):
    shutil.rmtree(data_dir)

dir_split(raw_dir, data_dir, val_size=0.2, test_size=0.2)

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

print("Train classes:", os.listdir(train_dir))

Train classes: ['j', 'u', 'q', 'e', 'c', 'o', 'f', 'raw', 'g', 'k', 'v', 'data', 'p', 't', 'w', 'd', 's', 'y', 'r', 'l', 'n', 'x', 'h', 'b', 'a', 'z', 'i', 'm']


In [ ]:
yolo_model = YOLO(model='yolo11x-cls.pt')
yolo_results = yolo_model.train(data=data_dir, epochs=5, imgsz=224)

Ultralytics 8.3.202 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/isl_dataset/data, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11x-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pos

In [ ]:
# Torch transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

train_ds = datasets.ImageFolder(train_dir, transform=transform)
val_ds = datasets.ImageFolder(val_dir, transform=transform)
test_ds = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit_model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=len(train_ds.classes),
    ignore_mismatched_sizes=True
)
vit_model.to(device)
optimizer = Adam(vit_model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
